In [1]:
# Import libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
import keras
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
# Loading the tokenizer to generate tokens and model to perform sentiment analysisasdadas
#tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
#model = AutoModelForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

In [3]:
# Define the path to save the tokenizer and model
#save_directory = '/content/drive/MyDrive/Final Year Project/twitter_roberta_sentiment'

# Save the tokenizer and model
#tokenizer.save_pretrained(save_directory)
#model.save_pretrained(save_directory)

In [4]:
# Define the path where the tokenizer and model are saved
load_directory = '/content/drive/MyDrive/Final Year Project/twitter_roberta_sentiment'

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(load_directory)
model = AutoModelForSequenceClassification.from_pretrained(load_directory)

In [5]:
# Initialize MinMaxScaler
scaler = MinMaxScaler()

In [6]:
# Start date, end date and the stock we want to predict
today = datetime.now()
start = datetime(today.year - 2, today.month, today.day)
end = datetime(today.year, today.month, today.day)
print(start)
print(end)

2022-06-10 00:00:00
2024-06-10 00:00:00


In [7]:
# Override how Pandas Datareader reads data
yf.pdr_override()

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


In [8]:
# Define the list of stocks you want to download
stocks = ['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS',
          'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BPCL.NS', 'BHARTIARTL.NS',
          'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS',
          'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS',
          'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ITC.NS',
          'INDUSINDBK.NS', 'INFY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LTIM.NS', 'LT.NS',
          'M&M.NS', 'MARUTI.NS', 'NTPC.NS', 'NESTLEIND.NS', 'ONGC.NS', 'POWERGRID.NS',
          'RELIANCE.NS', 'SBILIFE.NS', 'SHRIRAMFIN.NS', 'SBIN.NS', 'SUNPHARMA.NS',
          'TCS.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS',
          'TITAN.NS', 'ULTRACEMCO.NS', 'WIPRO.NS']

# Create an empty DataFrame to hold all the data
all_stock_data = pd.DataFrame()

# Loop through each stock and download the data
for stock in stocks:
    # Download data from Yahoo Finance
    df = yf.download(stock, start=start, end=end)

    # Ticker for searching news
    stockraw = stock[:-3]

    # Getting data using html requests
    link = f"https://news.google.com/search?q={stockraw}&hl=en-IN&gl=IN&ceid=IN%3Aen"
    req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')

    # Extract titles and dates
    titles = soup.find_all('a', class_='JtKRv')
    datetimes = soup.find_all('time', class_='hvbAAd')

    # Create lists to store extracted data
    dates_list = []
    titles_list = []

    for title, datetime_tag in zip(titles, datetimes):
        # Remove leading/trailing whitespace from title
        title_text = title.text.strip()
        datetime_str = datetime_tag['datetime']

        # Parse datetime string
        formatted_datetime = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S%z')

        # Append data to respective lists
        dates_list.append(formatted_datetime.date())
        titles_list.append(title_text)

    # Create DataFrame
    df2 = pd.DataFrame({'Date': dates_list, 'Headline': titles_list})

    # Finding the encodings for each of the headlines found
    encoded_headlines = []
    for headline in df2['Headline']:
        encoded_headline = tokenizer(headline, padding=True, truncation=True, return_tensors='pt')
        encoded_headlines.append(encoded_headline)

    # Finding the sentiment scores of each headline
    # Initialize an empty list to store the tweet sentiments
    headline_sentiments = []

    # Iterate over the encoded tweets
    for encoded_headline in encoded_headlines:
        # Pass the encoded tweet to the model
        output = model(**encoded_headline)

        # Extract the scores
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)

        # Calculate the sentiment score between -1 and 1
        sentiment_score = scores[2] - scores[0]

        # Append the sentiment score to the list
        headline_sentiments.append(sentiment_score)

    # Add the tweet sentiments as a new column to the DataFrame
    df2['Sentiments'] = headline_sentiments

    # Convert 'Date' column to datetime if it's not already
    df2['Date'] = pd.to_datetime(df2['Date'])

    # Group by 'Date' and calculate the average sentiment for each day
    average_sentiments = df2.groupby('Date')['Sentiments'].mean().reset_index()

    # Merge the dataframes on the 'Date' column with a left join to keep all rows from df
    merged_df = pd.merge(df, average_sentiments, on='Date', how='left')

    # Fill missing sentiment scores with 0
    merged_df['Sentiments'].fillna(0, inplace=True)

    # If you want to ensure that 'Sentiments' column is of numeric type
    merged_df['Sentiments'] = pd.to_numeric(merged_df['Sentiments'])

    # Isolating features for training
    cols = list(merged_df)[1:8]

    # Storing all data for scaling
    df_for_scaling = merged_df[cols]

    # Initialize the scaler
    scaler = MinMaxScaler(feature_range=(0, 1))

    # Fit and transform the numerical data
    scaled_numerical_data = scaler.fit_transform(df_for_scaling)

    # Create a new DataFrame for the scaled data
    df_scaled = df_for_scaling.copy()
    df_scaled[cols] = scaled_numerical_data

    # Add a column for the stock ticker
    df_scaled['Stock'] = stock

    # Add all data to the following dataframe
    all_stock_data = pd.concat([all_stock_data, df_scaled])

print(all_stock_data)

[*********************100%%**********************]  1 of 1 completed
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%****

         Open      High       Low     Close  Adj Close    Volume  Sentiments  \
0    0.344884  0.313330  0.378343  0.339727   0.339412  0.024234    0.454308   
1    0.339109  0.298554  0.344151  0.298220   0.297923  0.034739    0.454308   
2    0.372607  0.333843  0.339674  0.337489   0.337175  0.143612    0.454308   
3    0.354257  0.314964  0.379393  0.333502   0.333189  0.024412    0.454308   
4    0.349835  0.314130  0.347202  0.299314   0.299016  0.043164    0.454308   
..        ...       ...       ...       ...        ...       ...         ...   
486  0.490687  0.485012  0.493933  0.470557   0.472697  0.044699    0.256683   
487  0.467802  0.459897  0.358522  0.438704   0.440973  0.115144    0.166468   
488  0.434806  0.509047  0.480143  0.510171   0.512151  0.071241    0.367750   
489  0.515966  0.546854  0.554881  0.561028   0.562802  0.096620    0.533555   
490  0.596062  0.678639  0.641478  0.687098   0.688363  0.556647    0.633596   

           Stock  
0    ADANIENT.NS  
1

In [9]:
df_for_training = all_stock_data.copy()

df_for_training

,Open,High,Low,Close,Adj Close,Volume,Sentiments,Stock
0,0.344884,0.313330,0.378343,0.339727,0.339412,0.024234,0.454308,ADANIENT.NS
1,0.339109,0.298554,0.344151,0.298220,0.297923,0.034739,0.454308,ADANIENT.NS
2,0.372607,0.333843,0.339674,0.337489,0.337175,0.143612,0.454308,ADANIENT.NS
3,0.354257,0.314964,0.379393,0.333502,0.333189,0.024412,0.454308,ADANIENT.NS
4,0.349835,0.314130,0.347202,0.299314,0.299016,0.043164,0.454308,ADANIENT.NS
...,...,...,...,...,...,...,...,...
486,0.490687,0.485012,0.493933,0.470557,0.472697,0.044699,0.256683,WIPRO.NS
487,0.467802,0.459897,0.358522,0.438704,0.440973,0.115144,0.166468,WIPRO.NS
488,0.434806,0.509047,0.480143,0.510171,0.512151,0.071241,0.367750,WIPRO.NS
489,0.515966,0.546854,0.554881,0.561028,0.562802,0.096620,0.533555,WIPRO.NS


In [10]:
# Initialize OneHotEncoder for the 'Stock' column
one_hot_encoder = OneHotEncoder(sparse_output=False)
stock_encoded = one_hot_encoder.fit_transform(df_for_training[['Stock']])

# Create a DataFrame with the one-hot encoded columns
stock_encoded_df = pd.DataFrame(stock_encoded, columns=one_hot_encoder.get_feature_names_out(['Stock']))

# Concatenate the one-hot encoded columns with the original data
df_for_training = pd.concat([df_for_training.reset_index(drop=True), stock_encoded_df], axis=1)

# Drop the original 'Stock' column as it's now one-hot encoded
df_for_training.drop('Stock', axis=1, inplace=True)

In [11]:
stock_encoded

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [15]:
df_for_training

,Open,High,Low,Close,Adj Close,Volume,Sentiments,Stock_ADANIENT.NS,Stock_ADANIPORTS.NS,Stock_APOLLOHOSP.NS,...,Stock_SHRIRAMFIN.NS,Stock_SUNPHARMA.NS,Stock_TATACONSUM.NS,Stock_TATAMOTORS.NS,Stock_TATASTEEL.NS,Stock_TCS.NS,Stock_TECHM.NS,Stock_TITAN.NS,Stock_ULTRACEMCO.NS,Stock_WIPRO.NS
0,0.344884,0.313330,0.378343,0.339727,0.339412,0.024234,0.454308,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.339109,0.298554,0.344151,0.298220,0.297923,0.034739,0.454308,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.372607,0.333843,0.339674,0.337489,0.337175,0.143612,0.454308,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.354257,0.314964,0.379393,0.333502,0.333189,0.024412,0.454308,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.349835,0.314130,0.347202,0.299314,0.299016,0.043164,0.454308,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24543,0.490687,0.485012,0.493933,0.470557,0.472697,0.044699,0.256683,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24544,0.467802,0.459897,0.358522,0.438704,0.440973,0.115144,0.166468,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24545,0.434806,0.509047,0.480143,0.510171,0.512151,0.071241,0.367750,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24546,0.515966,0.546854,0.554881,0.561028,0.562802,0.096620,0.533555,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
df_for_training.to_csv('/content/drive/MyDrive/Datasets/Project/df_for_training.csv', index=False)

In [17]:
df_for_training.shape

(24548, 57)